# TMT Dashboard - Optimized for Apple M-Series

This notebook implements an institutional-grade LSTM-based stock prediction system,
**optimized for Apple Silicon (M1/M2/M3/M4)** with:

- **Metal Performance Shaders (MPS)** GPU acceleration
- **Rust-based compute kernels** for 10-50x faster feature engineering
- **Adaptive memory management** for variable RAM configurations
- **Zero-copy tensor operations** for efficiency

## Hardware Requirements
- Apple M-series Mac (M1, M2, M3, M4) - or any Mac with MPS support
- Minimum 8GB RAM (16GB+ recommended)
- PyTorch 2.0+ with MPS backend

## Installation
```bash
# Install Python dependencies
pip install -r tmt_optimized/requirements.txt

# Build Rust acceleration (optional but recommended)
cd tmt_optimized/rust_core && ./build.sh
```

In [ ]:
# ============================================================================
# SETUP: Import optimized modules and configure hardware
# ============================================================================

import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Add tmt_optimized to path
sys.path.insert(0, os.path.dirname(os.getcwd()))

# Standard imports
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import spearmanr
import yfinance as yf
import time

# TMT Optimized imports
try:
    from tmt_optimized import (
        MPSAccelerator,
        get_device,
        optimize_for_memory,
        OptimizedFeatureEngine,
        AdaptiveMemoryManager,
        MemoryConfig,
        OptimizedLSTM,
        MPSTrainer,
    )
    from tmt_optimized.lstm_model import ModelConfig, TrainingConfig
    TMT_AVAILABLE = True
except ImportError as e:
    print(f"TMT Optimized not available: {e}")
    print("Using fallback mode...")
    TMT_AVAILABLE = False

# Configure matplotlib
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (14, 8)

print("="*60)
print("TMT DASHBOARD - APPLE SILICON OPTIMIZED")
print("="*60)

In [ ]:
# ============================================================================
# HARDWARE DETECTION AND CONFIGURATION
# ============================================================================

print("\n" + "="*60)
print("HARDWARE CONFIGURATION")
print("="*60)

# Initialize MPS Accelerator
if TMT_AVAILABLE:
    accelerator = MPSAccelerator(prefer_mps=True, memory_fraction=0.8)
    device = accelerator.device
    device_info = accelerator.device_info
    
    print(f"\nDevice: {device_info.device_name}")
    print(f"Total Memory: {device_info.total_memory_gb:.1f} GB")
    print(f"Available Memory: {device_info.available_memory_gb:.1f} GB")
    print(f"Unified Memory: {device_info.unified_memory}")
    print(f"Float16 Support: {device_info.supports_float16}")
else:
    # Fallback device detection
    if torch.backends.mps.is_available():
        device = torch.device("mps")
        print("Using MPS (Metal Performance Shaders)")
    elif torch.cuda.is_available():
        device = torch.device("cuda")
        print(f"Using CUDA: {torch.cuda.get_device_name()}")
    else:
        device = torch.device("cpu")
        print("Using CPU")

# Initialize Memory Manager
if TMT_AVAILABLE:
    memory_config = MemoryConfig(
        max_memory_gb=device_info.total_memory_gb if TMT_AVAILABLE else 8.0,
        min_batch_size=8,
        max_batch_size=256,
        enable_checkpointing=True,
    )
    memory_manager = AdaptiveMemoryManager(memory_config)
    
    # Get recommended settings
    settings = optimize_for_memory(device_info.available_memory_gb)
    print(f"\nRecommended Settings for {device_info.available_memory_gb:.1f} GB:")
    for key, value in settings.items():
        print(f"  {key}: {value}")

print("\n" + "="*60)

In [ ]:
# ============================================================================
# DATA CONFIGURATION
# ============================================================================

# Target asset and date range
TARGET_TICKER = 'AAPL'
START_DATE = '2010-01-01'
END_DATE = '2025-12-01'

# Intermarket assets for analysis
INTERMARKET_TICKERS = {
    'target': TARGET_TICKER,
    'rates': '^TNX',           # 10-Year Treasury Yield
    'dollar': 'DX-Y.NYB',      # US Dollar Index
    'vix': '^VIX',             # Volatility Index
    'semis': 'SOXX',           # Semiconductor ETF
    'tech': 'XLK',             # Technology Sector
    'market': 'SPY'            # S&P 500 (benchmark)
}

print(f"Target: {TARGET_TICKER}")
print(f"Date Range: {START_DATE} to {END_DATE}")
print(f"Intermarket Assets: {len(INTERMARKET_TICKERS)}")

In [ ]:
# ============================================================================
# PHASE 1: DATA DOWNLOAD WITH PROGRESS TRACKING
# ============================================================================

print("\n" + "="*60)
print("PHASE 1: DOWNLOADING MULTI-ASSET DATA")
print("="*60)

data_dict = {}
download_times = {}

for name, ticker in INTERMARKET_TICKERS.items():
    print(f"\nDownloading {name.upper()}: {ticker}...", end=" ")
    start_time = time.time()
    
    try:
        df = yf.download(ticker, start=START_DATE, end=END_DATE, progress=False)
        
        # Handle MultiIndex columns
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)
        
        if not df.empty:
            data_dict[name] = df
            download_time = time.time() - start_time
            download_times[name] = download_time
            print(f"✓ {len(df)} records ({download_time:.2f}s)")
        else:
            print("✗ No data")
            
    except Exception as e:
        print(f"✗ Error: {str(e)}")

print(f"\nTotal download time: {sum(download_times.values()):.2f}s")

In [ ]:
# ============================================================================
# PHASE 1: DATA ALIGNMENT AND MERGING
# ============================================================================

print("\n" + "="*60)
print("ALIGNING AND MERGING DATA")
print("="*60)

# Find common dates
common_dates = None
for name, df in data_dict.items():
    if common_dates is None:
        common_dates = df.index
    else:
        common_dates = common_dates.intersection(df.index)

print(f"\nCommon trading dates: {len(common_dates)}")
print(f"Date range: {common_dates[0]} to {common_dates[-1]}")

# Align all dataframes
aligned_data = {}
for name, df in data_dict.items():
    aligned_df = df.loc[common_dates].copy()
    aligned_df.columns = [f"{name}_{col}" for col in aligned_df.columns]
    aligned_data[name] = aligned_df
    print(f"  {name.upper()}: {aligned_df.shape}")

# Merge all
merged_df = pd.concat(aligned_data.values(), axis=1)
print(f"\nMerged shape: {merged_df.shape}")

In [ ]:
# ============================================================================
# PHASE 2: OPTIMIZED FEATURE ENGINEERING
# ============================================================================

print("\n" + "="*60)
print("PHASE 2: OPTIMIZED FEATURE ENGINEERING")
print("="*60)

# Initialize optimized feature engine
if TMT_AVAILABLE:
    from tmt_optimized.feature_engine import FeatureConfig
    
    feature_config = FeatureConfig(
        ffd_d_values=[0.3, 0.4, 0.5],
        rolling_windows=[5, 10, 20, 50],
        use_rust=True,
        parallel=True,
    )
    engine = OptimizedFeatureEngine(feature_config)
else:
    engine = None

# Time the feature engineering
start_time = time.time()

# Extract target data
target_df = merged_df[[col for col in merged_df.columns if col.startswith('target_')]].copy()
target_df.columns = [col.replace('target_', '') for col in target_df.columns]

# Extract market data for neutralization
market_df = merged_df[[col for col in merged_df.columns if col.startswith('market_')]].copy()
market_df.columns = [col.replace('market_', '') for col in market_df.columns]

features = pd.DataFrame(index=merged_df.index)

# Calculate returns
returns = target_df['Close'].pct_change().values
log_prices = np.log(target_df['Close'].values)

print("\nEngineering features...")

In [ ]:
# ============================================================================
# FEATURE ENGINEERING: FFD (Fractional Differentiation)
# ============================================================================

print("\n1. Fractional Differentiation (FFD)...")
ffd_start = time.time()

if engine is not None:
    # Use optimized Rust/Numba implementation
    ffd_results = engine.frac_diff_ffd_batch(log_prices)
    for name, values in ffd_results.items():
        # Align lengths
        padded = np.concatenate([np.full(len(merged_df) - len(values), np.nan), values])
        features[name] = padded
else:
    # Fallback to pure NumPy
    for d in [0.3, 0.4, 0.5]:
        # Simple FFD implementation
        weights = [1.0]
        k = 1
        while True:
            w = -weights[-1] * (d - k + 1) / k
            if abs(w) < 1e-5:
                break
            weights.append(w)
            k += 1
        weights = np.array(weights)
        width = len(weights)
        result = np.convolve(log_prices, weights[::-1], mode='valid')
        padded = np.concatenate([np.full(len(merged_df) - len(result), np.nan), result])
        features[f'ffd_{d}'] = padded

ffd_time = time.time() - ffd_start
print(f"   FFD completed in {ffd_time:.3f}s")

In [ ]:
# ============================================================================
# FEATURE ENGINEERING: Microstructure & Technical Indicators
# ============================================================================

print("\n2. Microstructure Features...")
micro_start = time.time()

high = target_df['High'].values
low = target_df['Low'].values
close = target_df['Close'].values
volume = target_df['Volume'].values

if engine is not None:
    features['order_flow_imbalance'] = engine.compute_order_flow_imbalance(high, low, close, volume)
    features['quote_pressure'] = engine.compute_quote_pressure(high, low, close)
else:
    range_hl = high - low + 1e-10
    position = (close - low) / range_hl
    features['order_flow_imbalance'] = (2 * position - 1) * volume
    features['quote_pressure'] = 2 * position - 1

print(f"   Microstructure completed in {time.time() - micro_start:.3f}s")

print("\n3. Technical Indicators...")
tech_start = time.time()

if engine is not None:
    # RSI
    features['rsi'] = engine.compute_rsi(close)
    
    # MACD
    macd, signal, hist = engine.compute_macd(close)
    features['macd'] = macd
    features['macd_signal'] = signal
    features['macd_hist'] = hist
    
    # Bollinger Bands
    for period in [20, 50]:
        _, _, _, position = engine.compute_bollinger_bands(close, period)
        features[f'bb_position_{period}'] = position
    
    # Rolling statistics
    features['realized_vol'] = engine.rolling_std(returns, 20) * np.sqrt(252)
else:
    # Fallback implementations
    # RSI
    delta = pd.Series(close).diff()
    gain = delta.where(delta > 0, 0).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
    rs = gain / (loss + 1e-10)
    features['rsi'] = 100 - (100 / (1 + rs)).values
    
    # MACD
    ema12 = pd.Series(close).ewm(span=12).mean()
    ema26 = pd.Series(close).ewm(span=26).mean()
    features['macd'] = (ema12 - ema26).values
    features['macd_signal'] = pd.Series(features['macd']).ewm(span=9).mean().values
    features['macd_hist'] = features['macd'] - features['macd_signal']
    
    # Bollinger Bands
    for period in [20, 50]:
        ma = pd.Series(close).rolling(period).mean()
        std = pd.Series(close).rolling(period).std()
        features[f'bb_position_{period}'] = ((close - ma) / (std + 1e-10)).values
    
    features['realized_vol'] = pd.Series(returns).rolling(20).std().values * np.sqrt(252)

print(f"   Technical indicators completed in {time.time() - tech_start:.3f}s")

In [ ]:
# ============================================================================
# FEATURE ENGINEERING: Momentum & Volume
# ============================================================================

print("\n4. Momentum Features...")
for period in [5, 10, 20]:
    features[f'momentum_{period}'] = pd.Series(close).pct_change(period).values

print("\n5. Volume Features...")
if engine is not None:
    vol_mean = engine.rolling_mean(volume.astype(np.float64), 20)
else:
    vol_mean = pd.Series(volume).rolling(20).mean().values
features['volume_ratio'] = volume / (vol_mean + 1e-10)

print("\n6. Entropy Features...")
entropy_start = time.time()
if engine is not None:
    features['return_entropy'] = engine.rolling_entropy(returns, 20)
else:
    from scipy.stats import entropy
    def rolling_entropy(x, window=20, bins=10):
        result = np.full(len(x), np.nan)
        for i in range(window-1, len(x)):
            window_data = x[i-window+1:i+1]
            window_data = window_data[~np.isnan(window_data)]
            if len(window_data) >= 5:
                hist, _ = np.histogram(window_data, bins=bins, density=True)
                hist = hist[hist > 0]
                result[i] = entropy(hist)
        return result
    features['return_entropy'] = rolling_entropy(returns)
print(f"   Entropy completed in {time.time() - entropy_start:.3f}s")

In [ ]:
# ============================================================================
# FEATURE ENGINEERING: Intermarket Features
# ============================================================================

print("\n7. Intermarket Features...")

# Rates (Treasury Yield)
if 'rates_Close' in merged_df.columns:
    features['rates_diff'] = merged_df['rates_Close'].diff().values
    features['rates_diff_ma5'] = pd.Series(features['rates_diff']).rolling(5).mean().values

# VIX
if 'vix_Close' in merged_df.columns:
    features['vix_level'] = merged_df['vix_Close'].values
    features['vix_pct_change'] = merged_df['vix_Close'].pct_change().values
    vix_ma = merged_df['vix_Close'].rolling(252).mean()
    vix_std = merged_df['vix_Close'].rolling(252).std()
    features['vix_zscore'] = ((merged_df['vix_Close'] - vix_ma) / (vix_std + 1e-10)).values

# Tech relative strength
if 'tech_Close' in merged_df.columns and 'market_Close' in merged_df.columns:
    tech_ret = merged_df['tech_Close'].pct_change()
    spy_ret = merged_df['market_Close'].pct_change()
    features['tech_rel_strength'] = (tech_ret - spy_ret).values

# Semis momentum
if 'semis_Close' in merged_df.columns:
    features['semis_momentum_10'] = merged_df['semis_Close'].pct_change(10).values

print(f"   Intermarket features added")

In [ ]:
# ============================================================================
# FEATURE ENGINEERING: Target & Standardization
# ============================================================================

print("\n8. Target Variable...")
features['target_1d'] = target_df['Close'].pct_change(1).shift(-1).values

print("\n9. Standardizing Features...")
std_start = time.time()
for col in features.columns:
    if col not in ['target_1d'] and not col.startswith('regime_'):
        mean = features[col].expanding().mean()
        std = features[col].expanding().std()
        features[col] = (features[col] - mean) / (std + 1e-10)
print(f"   Standardization completed in {time.time() - std_start:.3f}s")

# Drop NaN rows
features_clean = features.dropna()

total_time = time.time() - start_time

print("\n" + "="*60)
print("FEATURE ENGINEERING COMPLETE")
print("="*60)
print(f"Total features: {len(features_clean.columns)}")
print(f"Total samples: {len(features_clean)}")
print(f"Total time: {total_time:.2f}s")
print(f"Date range: {features_clean.index[0]} to {features_clean.index[-1]}")

In [ ]:
# ============================================================================
# PHASE 3: DATA PREPARATION FOR LSTM
# ============================================================================

print("\n" + "="*60)
print("PHASE 3: PREPARING DATA FOR LSTM")
print("="*60)

# Configuration
SEQUENCE_LENGTH = settings.get('sequence_length', 30) if TMT_AVAILABLE else 30

# Separate features and target
feature_cols = [col for col in features_clean.columns if not col.startswith('target_')]
X = features_clean[feature_cols].values
y = features_clean['target_1d'].values

print(f"\nFeature matrix shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"Sequence length: {SEQUENCE_LENGTH}")

# Create sequences
seq_start = time.time()
if engine is not None:
    X_seq, y_seq = engine.create_sequences(X, y, SEQUENCE_LENGTH)
else:
    # Fallback
    n_sequences = len(X) - SEQUENCE_LENGTH
    X_seq = np.zeros((n_sequences, SEQUENCE_LENGTH, X.shape[1]))
    y_seq = np.zeros(n_sequences)
    for i in range(n_sequences):
        X_seq[i] = X[i:i+SEQUENCE_LENGTH]
        y_seq[i] = y[i+SEQUENCE_LENGTH]

print(f"Sequence creation time: {time.time() - seq_start:.3f}s")
print(f"X_seq shape: {X_seq.shape}")
print(f"y_seq shape: {y_seq.shape}")

# Train/test split (temporal)
split_idx = int(0.8 * len(X_seq))
X_train, X_test = X_seq[:split_idx], X_seq[split_idx:]
y_train, y_test = y_seq[:split_idx], y_seq[split_idx:]

print(f"\nTrain samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")

In [ ]:
# ============================================================================
# PHASE 4: MODEL TRAINING WITH MPS ACCELERATION
# ============================================================================

print("\n" + "="*60)
print("PHASE 4: TRAINING OPTIMIZED LSTM")
print("="*60)

if TMT_AVAILABLE:
    # Create optimized model
    model_config = ModelConfig(
        input_size=X_train.shape[2],
        hidden_size=settings.get('hidden_size', 64),
        num_layers=settings.get('num_layers', 2),
        dropout=0.3,
        use_layer_norm=True,
    )
    
    training_config = TrainingConfig(
        learning_rate=0.001,
        batch_size=settings.get('batch_size', 32),
        num_epochs=50,
        gradient_clip=1.0,
        early_stopping_patience=10,
        gradient_accumulation_steps=settings.get('gradient_accumulation', 1),
    )
    
    model = OptimizedLSTM(model_config)
    trainer = MPSTrainer(
        model=model,
        training_config=training_config,
        accelerator=accelerator,
        memory_manager=memory_manager,
    )
    
    # Train
    history = trainer.train(
        X_train=X_train.astype(np.float32),
        y_train=y_train.astype(np.float32),
        X_val=X_test.astype(np.float32),
        y_val=y_test.astype(np.float32),
        verbose=True,
    )
    
    # Generate predictions
    test_pred = trainer.predict(X_test.astype(np.float32))
    
else:
    # Fallback to basic PyTorch training
    import torch.nn as nn
    
    class BasicLSTM(nn.Module):
        def __init__(self, input_size, hidden_size=64, num_layers=2, dropout=0.3):
            super().__init__()
            self.lstm = nn.LSTM(input_size, hidden_size, num_layers,
                               batch_first=True, dropout=dropout)
            self.fc = nn.Linear(hidden_size, 1)
        
        def forward(self, x):
            lstm_out, _ = self.lstm(x)
            return self.fc(lstm_out[:, -1, :])
    
    model = BasicLSTM(X_train.shape[2]).to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    # Convert to tensors
    X_train_t = torch.FloatTensor(X_train).to(device)
    y_train_t = torch.FloatTensor(y_train).view(-1, 1).to(device)
    X_test_t = torch.FloatTensor(X_test).to(device)
    
    # Training loop
    history = {'train_loss': [], 'val_loss': []}
    batch_size = 32
    
    for epoch in range(50):
        model.train()
        epoch_loss = 0
        n_batches = 0
        
        for i in range(0, len(X_train_t), batch_size):
            batch_X = X_train_t[i:i+batch_size]
            batch_y = y_train_t[i:i+batch_size]
            
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
            n_batches += 1
        
        history['train_loss'].append(epoch_loss / n_batches)
        
        if (epoch + 1) % 10 == 0:
            print(f"Epoch [{epoch+1}/50], Loss: {epoch_loss/n_batches:.6f}")
    
    model.eval()
    with torch.no_grad():
        test_pred = model(X_test_t).cpu().numpy().flatten()

In [ ]:
# ============================================================================
# PHASE 5: EVALUATION AND VISUALIZATION
# ============================================================================

print("\n" + "="*60)
print("PHASE 5: EVALUATION")
print("="*60)

# Calculate metrics
if engine is not None:
    test_ic = engine.compute_ic(test_pred, y_test)
    strategy_returns = np.sign(test_pred) * y_test
    test_sharpe = engine.compute_sharpe_ratio(strategy_returns)
    test_max_dd = engine.compute_max_drawdown(strategy_returns)
else:
    test_ic = spearmanr(test_pred, y_test)[0]
    strategy_returns = np.sign(test_pred) * y_test
    test_sharpe = np.sqrt(252) * np.mean(strategy_returns) / (np.std(strategy_returns) + 1e-10)
    cumulative = np.cumprod(1 + strategy_returns)
    running_max = np.maximum.accumulate(cumulative)
    test_max_dd = np.min((cumulative - running_max) / running_max)

print(f"\nTest Results:")
print(f"  Information Coefficient (IC): {test_ic:.4f}")
print(f"  Sharpe Ratio: {test_sharpe:.4f}")
print(f"  Max Drawdown: {test_max_dd*100:.2f}%")
print(f"  Total Return: {np.sum(strategy_returns)*100:.2f}%")
print(f"  Win Rate: {(strategy_returns > 0).mean()*100:.2f}%")

In [ ]:
# ============================================================================
# VISUALIZATION
# ============================================================================

fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# 1. Training Loss
if TMT_AVAILABLE and 'train_loss' in history:
    axes[0, 0].plot(history['train_loss'], label='Train Loss', linewidth=2)
    axes[0, 0].plot(history['val_loss'], label='Val Loss', linewidth=2)
    axes[0, 0].legend()
else:
    axes[0, 0].plot(history['train_loss'], linewidth=2)
axes[0, 0].set_title('Training Loss', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].grid(True, alpha=0.3)

# 2. Predictions vs Actuals
axes[0, 1].scatter(test_pred, y_test, alpha=0.5, s=10)
axes[0, 1].plot([test_pred.min(), test_pred.max()],
                [test_pred.min(), test_pred.max()],
                'r--', linewidth=2, label='Perfect')
axes[0, 1].set_title(f'Predictions vs Actuals (IC={test_ic:.3f})', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Predicted')
axes[0, 1].set_ylabel('Actual')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. Cumulative Returns
cum_strategy = np.cumprod(1 + strategy_returns)
cum_buyhold = np.cumprod(1 + y_test)
axes[1, 0].plot(cum_strategy, label='Strategy', linewidth=2)
axes[1, 0].plot(cum_buyhold, label='Buy & Hold', linewidth=2, alpha=0.7)
axes[1, 0].set_title('Cumulative Returns', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Days')
axes[1, 0].set_ylabel('Cumulative Return')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# 4. Drawdown
running_max = np.maximum.accumulate(cum_strategy)
drawdown = (cum_strategy - running_max) / running_max
axes[1, 1].fill_between(range(len(drawdown)), drawdown, 0, alpha=0.7, color='red')
axes[1, 1].set_title(f'Drawdown (Max: {test_max_dd*100:.2f}%)', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Days')
axes[1, 1].set_ylabel('Drawdown')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# ============================================================================
# PERFORMANCE SUMMARY
# ============================================================================

print("\n" + "="*60)
print("PERFORMANCE SUMMARY")
print("="*60)

if TMT_AVAILABLE:
    print(f"\nHardware:")
    print(f"  Device: {device_info.device_name}")
    print(f"  Memory Used: {memory_manager.get_memory_stats().used_gb:.2f} GB")
    print(f"  Peak Memory: {memory_manager.get_memory_stats().peak_gb:.2f} GB")

print(f"\nModel Performance:")
print(f"  Information Coefficient: {test_ic:.4f}")
print(f"  Sharpe Ratio: {test_sharpe:.4f}")
print(f"  Max Drawdown: {test_max_dd*100:.2f}%")

print(f"\nOptimization Status:")
print(f"  Rust Core: {'Enabled' if (TMT_AVAILABLE and engine and engine.use_rust) else 'Disabled'}")
print(f"  MPS Acceleration: {'Enabled' if str(device) == 'mps' else 'Disabled'}")
print(f"  Adaptive Memory: {'Enabled' if TMT_AVAILABLE else 'Disabled'}")

print("\n" + "="*60)
print("OPTIMIZATION COMPLETE")
print("="*60)